# Dataset Cleaner

Import the necessary libs

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

Load the dataset

In [2]:
asd = pd.read_csv("assets/Amazon Sales data.csv")

Data Overview

In [3]:
def data_overview(data):
    print("Dataset Information:")
    print(data.info())
    print("\nDataset Description:")
    print(data.describe())
    print("\nMissing Values:")
    print(data.isnull().sum())

data_overview(asd)

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Region          100 non-null    object 
 1   Country         100 non-null    object 
 2   Item Type       100 non-null    object 
 3   Sales Channel   100 non-null    object 
 4   Order Priority  100 non-null    object 
 5   Order Date      100 non-null    object 
 6   Order ID        100 non-null    int64  
 7   Ship Date       100 non-null    object 
 8   Units Sold      100 non-null    int64  
 9   Unit Price      100 non-null    float64
 10  Unit Cost       100 non-null    float64
 11  Total Revenue   100 non-null    float64
 12  Total Cost      100 non-null    float64
 13  Total Profit    100 non-null    float64
dtypes: float64(5), int64(2), object(7)
memory usage: 11.1+ KB
None

Dataset Description:
           Order ID   Units Sold  Unit Price   Unit Cost  Total R

Data Cleaning and Transformation

In [8]:
def clean_and_transform_data(data):
    data["Order Date"] = pd.to_datetime(data["Order Date"])
    data["Ship Date"] = pd.to_datetime(data["Ship Date"])
    data["Shipping Time (days)"] = (data["Ship Date"] - data["Order Date"]).dt.days
    data["Total Profit"] = data["Total Revenue"] - data["Total Cost"]
    data["Average Order Value (AOV)"] = (
        data["Total Revenue"] / data["Order ID"].nunique()
    )
    data["Profit Margin"] = (data["Total Profit"] / data["Total Revenue"]) * 100
    return data

asdc = clean_and_transform_data(asd)

Convert 'Order Date' and 'Ship Date' to datetime

In [9]:
asdc['Order Date'] = pd.to_datetime(asdc['Order Date'])
asdc['Ship Date'] = pd.to_datetime(asdc['Ship Date'])

Check for any duplicates based on 'Order ID'

In [11]:
asdc = asdc.drop_duplicates(subset='Order ID')
asdc

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,Shipping Time (days),Average Order Value (AOV),Profit Margin
0,Australia and Oceania,Tuvalu,Baby Food,Offline,H,2010-05-28,669165933,2010-06-27,9925,255.28,159.42,2533654.00,1582243.50,951410.50,30,25336.5400,37.550924
1,Central America and the Caribbean,Grenada,Cereal,Online,C,2012-08-22,963881480,2012-09-15,2804,205.70,117.11,576782.80,328376.44,248406.36,24,5767.8280,43.067574
2,Europe,Russia,Office Supplies,Offline,L,2014-05-02,341417157,2014-05-08,1779,651.21,524.96,1158502.59,933903.84,224598.75,6,11585.0259,19.386987
3,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Online,C,2014-06-20,514321792,2014-07-05,8102,9.33,6.92,75591.66,56065.84,19525.82,15,755.9166,25.830654
4,Sub-Saharan Africa,Rwanda,Office Supplies,Offline,L,2013-02-01,115456712,2013-02-06,5062,651.21,524.96,3296425.02,2657347.52,639077.50,5,32964.2502,19.386987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sub-Saharan Africa,Mali,Clothes,Online,M,2011-07-26,512878119,2011-09-03,888,109.28,35.84,97040.64,31825.92,65214.72,39,970.4064,67.203514
96,Asia,Malaysia,Fruits,Offline,L,2011-11-11,810711038,2011-12-28,6267,9.33,6.92,58471.11,43367.64,15103.47,47,584.7111,25.830654
97,Sub-Saharan Africa,Sierra Leone,Vegetables,Offline,C,2016-06-01,728815257,2016-06-29,1485,154.06,90.93,228779.10,135031.05,93748.05,28,2287.7910,40.977541
98,North America,Mexico,Personal Care,Offline,M,2015-07-30,559427106,2015-08-08,5767,81.73,56.67,471336.91,326815.89,144521.02,9,4713.3691,30.661936


Fill missing values as requirement for numerical columns like 'Units Sold' into with median

In [12]:
asdc['Units Sold'] = asdc['Units Sold'].fillna(asdc['Units Sold'].median())

For categorical columns like 'Order Priority', filled with mode

In [13]:
asdc['Order Priority'] = asdc['Order Priority'].fillna(asdc['Order Priority'].mode()[0])

Ensure correct data types

In [14]:
asdc['Unit Price'] = asdc['Unit Price'].astype(float)
asdc['Unit Cost'] = asdc['Unit Cost'].astype(float)
asdc['Total Revenue'] = asdc['Total Revenue'].astype(float)
asdc['Total Cost'] = asdc['Total Cost'].astype(float)
asdc['Total Profit'] = asdc['Total Profit'].astype(float)

Remove outliers by a simple method using z-score for 'Total Profit'

In [15]:
asdc = asdc[(np.abs(stats.zscore(asdc['Total Profit'])) < 3)]

Save the cleaned dataset with UTF-8 encoding

In [17]:
asdc.to_csv('assets/cleaned_dataset.csv', index=False, encoding='utf-8')